In [1]:
!pip install transformers datasets rouge-score -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from transformers import  BartTokenizer, BartForConditionalGeneration
from datasets import load_dataset
from rouge_score import rouge_scorer

In [3]:
def load_model(model_name):
  tokenizer = BartTokenizer.from_pretrained(model_name)
  model = BartForConditionalGeneration.from_pretrained(model_name)
  return model, tokenizer

In [4]:
dataset = load_dataset("cnn_dailymail", "3.0.0", split="test")
texts = dataset['article'][6:11]
references = dataset['highlights'][6:11]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [5]:
def summarize(text, model, tokenizer):
    input_ids = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        output = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return summary

In [6]:
def calculate_rouge(summaries, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = { 'rouge1': 0, 'rouge2': 0, 'rougeL': 0 }
    i=0
    for summary, reference in zip(summaries, references):
        i+=1

        score = scorer.score(reference, summary)
        print(f'rouge1 score of summary{i}:{score["rouge1"].fmeasure}')
        print(f'rouge2 score of summary{i}:{score["rouge2"].fmeasure}')
        print(f'rougeL score of summary{i}:{score["rougeL"].fmeasure}')
        scores['rouge1'] += score['rouge1'].fmeasure
        scores['rouge2'] += score['rouge2'].fmeasure
        scores['rougeL'] += score['rougeL'].fmeasure
        ws.cell(row=i+1, column=4, value=scores['rouge1'])
        ws.cell(row=i+1, column=5, value=scores['rouge2'])
        ws.cell(row=i+1, column=6, value=scores['rougeL'])

    num_samples = len(summaries)
    scores = { k: v / num_samples for k, v in scores.items() }
    for j, val in enumerate(scores.values()):
        ws.cell(row=i+2, column=j+4, value=val)
    return scores

In [7]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(summaries, references):

    bleu_scores = []
    smoothie = SmoothingFunction().method4
    i=0
    for summary, reference in zip(summaries, references):
        i+=1
        reference_tokens = [reference.split()]
        summary_tokens = summary.split()

        score = sentence_bleu(reference_tokens, summary_tokens,weights=(0.5,0.5,0,0) ,smoothing_function=smoothie)
        bleu_scores.append(score)
        print(f"BLEU score of summary{i}: {score}")
        ws.cell(row=i+1, column=7, value=score)

    average_bleu = sum(bleu_scores) / len(bleu_scores)
    ws.cell(row=i+2, column=7, value=average_bleu)
    return average_bleu



In [8]:
def run_summarization(model_name):
  model, tokenizer = load_model(model_name)

  summaries = [summarize(text, model, tokenizer) for text in texts]

  for i, summary in enumerate(summaries):
    print(f"\nGenerated Summary {i+1} ({model_name}):\n", summary)

  column=['Text',"reference Summary","Summary","rouge 1","rouge 2","rouge L","Bleu score"]
  for i,val in enumerate(column):
      ws.cell(row=1, column=i+1, value=val)
  for i, text in enumerate(texts):
      ws.cell(row=i+2, column=1, value=text)
  for i, summary in enumerate(summaries):
      ws.cell(row=i+2, column=3, value=summary)
  for i, ref in enumerate(references):
      ws.cell(row=i+2, column=2, value=ref)

  rouge_scorers = calculate_rouge(summaries, references)
  print(f"\nROUGE Scores for {model_name}:", rouge_scorers)
  bleu_score = calculate_bleu(summaries, references)
  print(f"Average BLEU score: {bleu_score}")

In [9]:
!pip install openpyxl
import openpyxl
from openpyxl import Workbook, load_workbook

In [10]:
wb = load_workbook('/content/drive/My Drive/summary_results.xlsx')

In [11]:
ws = wb.create_sheet(title="bart-large-cnn")

In [12]:
run_summarization("facebook/bart-large-cnn")


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]


Generated Summary 1 (facebook/bart-large-cnn):
 The number of executions worldwide has gone down by almost 22% on the previous year. At least 607 people were executed around the world in 2014, compared to 778 in 2013. The annual report catalogs the use of state-sanctioned killing as a punitive measure across the globe.

Generated Summary 2 (facebook/bart-large-cnn):
 Coroner's preliminary assessment is there was no foul play involved in the death. Andrew Getty, 47, had "several health issues," police say. He is the grandson of oil tycoon J. Paul Getty, who died in 1976. Gordon Getty spearheaded the controversial sale of Getty to Texaco for $10 billion in 1984.

Generated Summary 3 (facebook/bart-large-cnn):
 Maysak gained super typhoon status just a few days ago. It has since lost steam and is now classified as a tropical storm. Authorities are barring outdoor activities like swimming, surfing, diving and boating. It's expected to make landfall Sunday morning on the southeastern coast

In [13]:
ws = wb.create_sheet(title="bart-large")

In [14]:
run_summarization("facebook/bart-large")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]


Generated Summary 1 (facebook/bart-large):
 (CNN)Governments around the world are using the threat of terrorism -- real or perceived -- to advance executions, Amnesty International alleges in its annual report on the death penalty. The annual report catalogs the use of state-sanctioned killing as a punitive measure across the globe, and this year's edition contains some mixed findings. On one hand, the number of executions worldwide has gone down by almost 22% on the previous year. Amnesty's figures do not include statistics on executions carried out in China, where information on the practice is regarded as a state secret. Belarus and Vietnam, too, do not release data on death penalty cases. On the other hand, Amnesty's report notes a sharp increase in executions in Sub-Saharan

Generated Summary 2 (facebook/bart-large):
 (CNN)Andrew Getty, one of the heirs to billions of oil money, appears to have died of natural causes, a Los Angeles Police Department spokesman said. The coroner's 

In [15]:
ws = wb.create_sheet(title="bart-base")

In [16]:
run_summarization("facebook/bart-base")

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]


Generated Summary 1 (facebook/bart-base):
 (CNN)Governments around the world are using the threat of terrorism -- real or perceived -- to advance executions, Amnesty International alleges in its annual report on the death penalty. "The dark trend of governments using the death Penalty in a futile attempt to tackle real or imaginary threats to state security and public safety was stark last year," said Salil Shetty, Amnesty's Secretary General in a release. "It is shameful that so many states around the globe are essentially playing with people's lives -- putting people to death for 'terrorism' or to quell internal instability on the ill-conceived premise of deterrence." The report, "Death Sentences and Executions 2014," cites the example of Pakistan lifting a six-year moratorium on

Generated Summary 2 (facebook/bart-base):
 (CNN)Andrew Getty, one of the heirs to billions of oil money, appears to have died of natural causes, a Los Angeles Police Department spokesman said. The coroner'

In [17]:
wb.save('/content/drive/My Drive/summary_results.xlsx')